In [1]:
!pip uninstall opencv-python -y
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16 --force-reinstall

Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 25.0MB 7.3MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6MB 161kB/s 
     |████████████████████████████████| 14.5MB 155kB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!tar xvzf '/content/gdrive/My Drive/101_ObjectCategories.tar.gz' 

Streaming output truncated to the last 5000 lines.
101_ObjectCategories/chair/image_0005.jpg
101_ObjectCategories/chair/image_0006.jpg
101_ObjectCategories/chair/image_0007.jpg
101_ObjectCategories/chair/image_0008.jpg
101_ObjectCategories/chair/image_0010.jpg
101_ObjectCategories/chair/image_0011.jpg
101_ObjectCategories/chair/image_0012.jpg
101_ObjectCategories/chair/image_0013.jpg
101_ObjectCategories/chair/image_0014.jpg
101_ObjectCategories/chair/image_0016.jpg
101_ObjectCategories/chair/image_0017.jpg
101_ObjectCategories/chair/image_0018.jpg
101_ObjectCategories/chair/image_0019.jpg
101_ObjectCategories/chair/image_0020.jpg
101_ObjectCategories/chair/image_0022.jpg
101_ObjectCategories/chair/image_0023.jpg
101_ObjectCategories/chair/image_0024.jpg
101_ObjectCategories/chair/image_0025.jpg
101_ObjectCategories/chair/image_0026.jpg
101_ObjectCategories/chair/image_0028.jpg
101_ObjectCategories/chair/image_0029.jpg
101_ObjectCategories/chair/image_0030.jpg
101_ObjectCategories/chai

In [6]:
import os
import cv2
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import math
import copy
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [7]:
np.random.seed(0)

In [8]:
rootdir =  '/content/101_ObjectCategories'
categories = os.listdir(rootdir)
if '.DS_Store' in categories: categories.remove('.DS_Store')

In [9]:
train_data = []
train_label = []
test_data = []
test_label = []

for cat in categories:
    files = os.listdir(rootdir + '/' + str(cat))
    for i in range(len(files)):
        if i >= 80:
          break
        name = files[i]
        img = cv2.imread(rootdir + '/' + str(cat) + '/' + str(name), 0)
        if i <= 29:
            train_data.append(img)
            train_label.append(cat)
        else:
            test_data.append(img)
            test_label.append(cat)

In [10]:
def computeSIFT(data):
    x = []
    for i in range(len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 24
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
        dense_feat = sift.compute(img, kp)
        x.append(dense_feat[1])
    return x

x_train = computeSIFT(train_data)
x_test = computeSIFT(test_data)

all_train_desc = []
for i in range(len(x_train)):
    for j in range(x_train[i].shape[0]):
        all_train_desc.append(x_train[i][j,:])

all_train_desc = np.array(all_train_desc)

In [11]:
from sklearn.cluster import MiniBatchKMeans
clusters = MiniBatchKMeans(n_clusters=200, max_iter=10000, batch_size=1000)
clusters.fit(all_train_desc)

MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
                init_size=None, max_iter=10000, max_no_improvement=10,
                n_clusters=200, n_init=3, random_state=None,
                reassignment_ratio=0.01, tol=0.0, verbose=0)

In [12]:
def getImageFeaturesSPM(L, img, clusters, k):
    W = img.shape[1]
    H = img.shape[0]   
    h = []
    for l in range(L+1):
      w_step = math.floor(W/(2**l))
      h_step = math.floor(H/(2**l))
      x, y = 0, 0
      for _ in range(2**l):
        x = 0
        for _ in range(2**l):                
          desc = extractSIFT(img[y:y+h_step, x:x+w_step])                
          predict = clusters.predict(desc)
          histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
          weight = 2**(l-L)
          h.append(weight*histo)
          x = x + w_step
        y = y + h_step
            
    hist = np.array(h).ravel().astype('float64')
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist

def getImageFeaturesBOW(l, img, clusters, k):
    W = img.shape[1]
    H = img.shape[0]   
    h = []
    w_step = math.floor(W/(2**l))
    h_step = math.floor(H/(2**l))
    x, y = 0, 0
    for _ in range(2**l):
      x = 0
      for _ in range(2**l):
        desc = extractSIFT(img[y:y+h_step, x:x+w_step]) 
        predict = clusters.predict(desc)
        histo = np.bincount(predict, minlength=k).reshape(1,-1).ravel()
        h.append(histo)
        x = x + w_step
      y = y + h_step
            
    hist = np.array(h).ravel().astype('float64')
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist

def getHistogramSPM(L, data, clusters, k):    
    x = []
    for i in range(len(data)):        
        hist = getImageFeaturesSPM(L, data[i], clusters, k)        
        x.append(hist)
    return np.array(x)


def getHistogramBOW(L, data, clusters, k):    
    x = []
    for i in range(len(data)):        
        hist = getImageFeaturesBOW(L, data[i], clusters, k)        
        x.append(hist)
    return np.array(x)

In [13]:
L = 4
M = 200

train_hist_spm = []
test_hist_spm = []

def extractSIFT(img):
    sift = cv2.xfeatures2d.SIFT_create()
    step_size = 24
    kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
    dense_feat = sift.compute(img, kp)
    return dense_feat[1]

for l in range(L):
  print(l)
  train_hist = getHistogramSPM(l, train_data, clusters, 200)
  test_hist = getHistogramSPM(l, test_data, clusters, 200)
  train_hist_spm.append(train_hist)
  test_hist_spm.append(test_hist)

0
1
2
3


In [14]:

train_hist_bow = []
test_hist_bow = []

for l in range(L):
  print(l)
  train_hist = getHistogramBOW(l, train_data, clusters, 200)
  test_hist = getHistogramBOW(l, test_data, clusters, 200)
  train_hist_bow.append(train_hist)
  test_hist_bow.append(test_hist)


0
1
2
3


In [23]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

def svc_param_selection(X_train, y_train, X_test, y_test):
    for l in range(len(X_train)):
      Cs = [0.007]
      param_grid = {'C': Cs}
      grid_search = GridSearchCV(LinearSVC(random_state=0), param_grid)
      grid_search.fit(X_train[l], y_train)
      best_est = grid_search.best_estimator_
      yhat = best_est.predict(X_test[l])
      acc = accuracy_score(y_test, yhat)
      print(acc)

svc_param_selection(train_hist_bow , train_label, test_hist_bow, test_label)

0.4020033388981636
0.42070116861435725
0.462771285475793
0.4617696160267112


In [24]:
svc_param_selection(train_hist_spm , train_label, test_hist_spm, test_label)

0.4020033388981636
0.4651085141903172
0.5225375626043406
0.5499165275459098


In [25]:
def get_accuracy(X_train, y_train, X_test, y_test):
  accuracy = []
  for l in range(len(X_train)):
    model = SVC()
    # clf = LinearSVC(random_state=0, max_iter=100000)
    ovr = OneVsRestClassifier(model)
    ovr.fit(X_train[l], y_train)
    yhat = ovr.predict(X_test[l])
    acc = accuracy_score(y_test, yhat)
    accuracy.append(acc)
    print(acc)
  print("\n")
  return accuracy

acc_bow = get_accuracy(train_hist_bow , train_label, test_hist_bow, test_label)
acc_spm = get_accuracy(train_hist_spm , train_label, test_hist_spm, test_label)

0.42237061769616024
0.4604340567612688
0.48146911519198665
0.4651085141903172


0.42237061769616024
0.48981636060100164
0.5275459098497496
0.5308848080133556


